In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import jax
import jax.numpy as jnp
import functools
import pickle
from operator import add
import matplotlib as mpl
from wazy.utils import *
from wazy.mlp import *
from jax_unirep import get_reps
import wazy
import os

In [ ]:
AA_list = [
    "A",
    "R",
    "N",
    "D",
    "C",
    "Q",
    "E",
    "G",
    "H",
    "I",
    "L",
    "K",
    "M",
    "F",
    "P",
    "S",
    "T",
    "W",
    "Y",
    "V",
    "B",
    "Z",
    "X",
    "*",
]
blosum92 = np.loadtxt("./blosum62.txt", dtype="i", delimiter=" ")

avg92 = jnp.sum(blosum92) / 24 / 24
sum92 = 0.0
for row in blosum92:
    for aa in row:
        sum92 += (aa - avg92) ** 2
std92 = jnp.sqrt(sum92 / 24 / 24)


def blosum(seq1, seq2):
    seqlist1 = list(seq1)
    seqlist2 = list(seq2)
    score = 0.0
    for i in range(len(seqlist1)):
        idx1 = AA_list.index(seqlist1[i])
        idx2 = AA_list.index(seqlist2[i])
        score += (blosum92[idx1][idx2] - avg92) / std92
        # jax.nn.sigmoid(score/len(seqlist1))
    return score / len(seqlist1)

In [ ]:
target_seq = "TARGETPEPTIDE"
key = jax.random.PRNGKey(0)

In [ ]:
with open("../10kseqs.txt") as f:
    readfile = f.readlines()
    random_seqs = f"{readfile[0]}".split(" ")[:-1]


def get_blosum_labels(seqs):
    labels = []
    for seq in seqs:
        labels.append(blosum(target_seq, seq))
    labels = np.array(labels)

    return labels


def get_count_labels(seqs):
    return get_aanum(seqs)[:, 0]


def get_aanum(seqs):
    aa_count = []
    for seq in seqs:
        seq_list = list(seq)
        aa_num = [float(seq_list.count(aa)) for aa in AA_list]
        aa_count.append(aa_num)
    aa_count = jnp.array(aa_count)
    return aa_count


def get_flat_ohc(seqs):
    return jnp.array([encode_seq(list(s)).flatten() for s in seqs])


def get_ohc(seqs):
    return jnp.array([encode_seq(list(s)) for s in seqs])


batch_size = 8


def get_results(key, params, rep_list):
    means = []
    stds = []
    # need batch
    for i in range(0, len(rep_list) // batch_size):
        # for rep in rep_list:
        batch_reps = rep_list[i * batch_size : (i + 1) * batch_size]
        yhat = forward_t.apply(params, key, batch_reps)
        # print(yhat.shape)
        means.append(yhat[0])
        stds.append(yhat[1])
    return np.array(means), np.array(stds)


def get_single_results(key, params, rep_list):
    yhats = []
    for i in range(0, len(rep_list) // batch_size):
        # need batch
        # for rep in rep_list:
        batch_reps = rep_list[i * batch_size : (i + 1) * batch_size]
        yhat = naive_forward_t.apply(params, key, batch_reps)
        yhats.append(yhat)
    return np.array(yhats)

In [ ]:
validation_seqs = [random.choice(random_seqs) for i in range(50)]
validation_ohc = get_ohc(validation_seqs)
validation_labels = get_blosum_labels(validation_seqs)
test_seqs = [random.choice(random_seqs) for i in range(500)]
test_ohc = get_ohc(test_seqs)
test_labels = get_blosum_labels(test_seqs)
train_seqs = [random.choice(random_seqs) for i in range(100)]
train_ohc = get_ohc(train_seqs)
train_labels = get_blosum_labels(train_seqs)

In [ ]:
train_ohc.reshape(100, -1).shape

In [ ]:
from sklearn.linear_model import LinearRegression

linear_model = LinearRegression()
linear_model.fit(train_ohc.reshape(100, -1), train_labels)

In [ ]:
yhat = linear_model.predict(test_ohc.reshape(500, -1))

In [ ]:
plt.scatter(test_labels, yhat, c="lightskyblue", alpha=0.3)
# plt.errorbar(test_y, yhat, yerr=std, fmt='None', alpha=0.3)
plt.plot([-0.5, 1.0], [-0.5, 1.0], color="lightsalmon", linewidth=2)
plt.title("Parity Plot")
plt.xlim([-0.5, 1.0])
plt.ylim([-0.5, 1.0])
plt.xlabel("Ground truth")
plt.ylabel("Predictions")
plt.text(
    min(test_y) + 0.1,
    max(test_y) - 0.2,
    f"correlation = {np.corrcoef(test_labels, yhat)[0,1]:.3f}",
)
plt.text(
    min(test_y) + 0.1,
    max(test_y) - 0.3,
    f"loss = {np.sqrt(np.mean((test_labels - yhat)**2)):.3f}",
)
plt.grid()